In [12]:
import pandas as pd

In [13]:
df = pd.read_parquet("SPX_Options.parquet")
df['overnight'] = df['delta'].diff()
df

,date,exp,strike,call,put,spot,SPY,tte,straddle,r,q,iv,delta,overnight,straddle_trade_cash_flow,spot_change,delta_hedging_cash_flow,cumulative_straddle_cash_flow,cumulative_delta_cash_flow,pnl
index,,,,,,,,,,,,,,,,,,,,
0,2023-03-28,2023-04-21,3500,486.80,3.32,3971.270020,395.600006,0.095238,490.12,0.0484,0.0171,0.227145,0.938315,NaN,-490.120000,0.0,-0.0,-4.901200e+02,-0.000000,-4.901200e+02
239,2023-03-28,2023-11-17,4000,274.42,226.99,3971.270020,395.600006,0.928571,501.41,0.0484,0.0171,0.154685,0.172271,-0.766044,-501.410000,0.0,-0.0,-9.915300e+02,-0.000000,-9.915300e+02
2,2023-03-28,2023-04-21,3655,334.62,10.02,3971.270020,395.600006,0.095238,344.64,0.0484,0.0171,0.207053,0.830254,0.657983,-344.640000,0.0,-0.0,-1.336170e+03,-0.000000,-1.336170e+03
3,2023-03-28,2023-04-21,3675,319.27,10.95,3971.270020,395.600006,0.095238,330.22,0.0484,0.0171,0.201752,0.813264,-0.016990,-330.220000,0.0,-0.0,-1.666390e+03,-0.000000,-1.666390e+03
4,2023-03-28,2023-04-21,3720,268.27,12.70,3971.270020,395.600006,0.095238,280.97,0.0484,0.0171,0.186539,0.774742,-0.038522,-280.970000,0.0,-0.0,-1.947360e+03,-0.000000,-1.947360e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90744,2024-04-05,2025-12-19,4100,1430.23,110.68,5204.339844,518.429993,2.472222,1540.91,0.0532,0.0136,0.209225,0.739520,-0.019247,-1540.910000,0.0,-0.0,-4.350548e+07,-1381.417629,-4.350686e+07
90745,2024-04-05,2025-12-19,6000,233.25,651.20,5204.339844,518.429993,2.472222,884.45,0.0532,0.0136,0.103935,-0.145539,-0.885059,-884.450000,0.0,0.0,-4.350637e+07,-1381.417629,-4.350775e+07
90746,2024-04-05,2028-12-15,5100,1278.56,458.82,5204.339844,518.429993,6.805556,1737.38,0.0532,0.0136,0.221667,0.520149,0.665688,-1737.380000,0.0,-0.0,-4.350810e+07,-1381.417629,-4.350948e+07


In [7]:
df = df.sort_values(by='date')

# Add a column for the straddle trade cash flow - initial premium paid is negative because it's a cash outflow
df['straddle_trade_cash_flow'] = -(df['straddle'])

df['spot_change'] = df['spot'].diff().fillna(0)

df['delta_hedging_cash_flow'] = -df['delta'] * df['spot_change'] 

#at the end cash flow
last_row = df.iloc[-1]
straddle_exercise_value = max(last_row['spot'] - last_row['strike'], 0) + max(last_row['strike'] - last_row['spot'], 0)
straddle_exercise_cash_flow = straddle_exercise_value - last_row['straddle_trade_cash_flow']
df.at[df.index[-1], 'straddle_trade_cash_flow'] += straddle_exercise_cash_flow

# Calculate cumulative cash flows
df['cumulative_straddle_cash_flow'] = df['straddle_trade_cash_flow'].cumsum()
df['cumulative_delta_cash_flow'] = df['delta_hedging_cash_flow'].cumsum()

# P&L
df['pnl'] = df['cumulative_straddle_cash_flow'] + df['cumulative_delta_cash_flow']

df


,date,exp,strike,call,put,spot,SPY,tte,straddle,r,q,iv,delta,overnight,straddle_trade_cash_flow,spot_change,delta_hedging_cash_flow,cumulative_straddle_cash_flow,cumulative_delta_cash_flow,pnl
0,2023-03-28,2023-04-21,3500,486.80,3.32,3971.270020,395.600006,0.095238,490.12,0.0484,0.0171,0.227145,0.938315,NaN,-490.120000,0.0,-0.0,-4.901200e+02,-0.000000,-4.901200e+02
239,2023-03-28,2023-11-17,4000,274.42,226.99,3971.270020,395.600006,0.928571,501.41,0.0484,0.0171,0.154685,0.172271,-0.122993,-501.410000,0.0,-0.0,-9.915300e+02,-0.000000,-9.915300e+02
2,2023-03-28,2023-04-21,3655,334.62,10.02,3971.270020,395.600006,0.095238,344.64,0.0484,0.0171,0.207053,0.830254,-0.023649,-344.640000,0.0,-0.0,-1.336170e+03,-0.000000,-1.336170e+03
3,2023-03-28,2023-04-21,3675,319.27,10.95,3971.270020,395.600006,0.095238,330.22,0.0484,0.0171,0.201752,0.813264,-0.016990,-330.220000,0.0,-0.0,-1.666390e+03,-0.000000,-1.666390e+03
4,2023-03-28,2023-04-21,3720,268.27,12.70,3971.270020,395.600006,0.095238,280.97,0.0484,0.0171,0.186539,0.774742,-0.038522,-280.970000,0.0,-0.0,-1.947360e+03,-0.000000,-1.947360e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90744,2024-04-05,2025-12-19,4100,1430.23,110.68,5204.339844,518.429993,2.472222,1540.91,0.0532,0.0136,0.209225,0.739520,-0.019247,-1540.910000,0.0,-0.0,-4.350548e+07,-1381.417629,-4.350686e+07
90745,2024-04-05,2025-12-19,6000,233.25,651.20,5204.339844,518.429993,2.472222,884.45,0.0532,0.0136,0.103935,-0.145539,-0.885059,-884.450000,0.0,0.0,-4.350637e+07,-1381.417629,-4.350775e+07
90746,2024-04-05,2028-12-15,5100,1278.56,458.82,5204.339844,518.429993,6.805556,1737.38,0.0532,0.0136,0.221667,0.520149,0.665688,-1737.380000,0.0,-0.0,-4.350810e+07,-1381.417629,-4.350948e+07
90711,2024-04-05,2024-12-20,5550,172.05,348.26,5204.339844,518.429993,1.027778,520.31,0.0532,0.0136,0.108666,-0.124724,-0.032602,-520.310000,0.0,0.0,-4.350862e+07,-1381.417629,-4.351000e+07


In [14]:
df.to_parquet("SPX_Options.parquet2", engine="fastparquet")